In [87]:
import numpy as np
import pandas as pd
from scipy.sparse import csgraph
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.tree import ExtraTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier

In [3]:
def labelencode_dataframe(dataframe_columns, classlist, column_names):
    le = LabelEncoder().fit(classlist)
    if len(dataframe_columns.shape) == 1:
        df_new = pd.DataFrame(le.transform(dataframe_columns), columns=column_names)
    else:
        df_new = dataframe_columns.apply(le.transform)
    return df_new

In [4]:
def dataframe_tolist(dataframe):
    ret_list = []
    headers = list(dataframe)
    for header in headers:
        ret_list.extend(list(set(dataframe[header])))
    return list(set(ret_list))

In [48]:
data = pd.read_csv('psl_data_wStats.csv', header=0, delimiter=',')

In [51]:
#Separating all features for encoding
teams = list(set(sum([list(data[['team1']]), list(data['team2'])], [])))
venues = list(set((data['venue'])))
toss_decisions = list(set(data['toss_decision']))
#encoding.
team1_new = labelencode_dataframe(data['team1'], teams, ['team1'])
team2_new = labelencode_dataframe(data['team2'], teams, ['team2'])
toss_winner_new = labelencode_dataframe(data['toss_winner'], teams, ['toss_winner'])
winner_new = labelencode_dataframe(data['winner'], teams, ['winner'])
venue_new = labelencode_dataframe(data['venue'], venues, ['venue'])
toss_decision_new = labelencode_dataframe(data['toss_decision'], toss_decisions, ['toss_decision'])

#New Encoded Data
data_new = pd.concat([team1_new, team2_new, data.iloc[:,2:12], venue_new, toss_winner_new, toss_decision_new], axis=1)
#Features
X = data_new.iloc[:,0:27]
#Labels
Y = pd.DataFrame(np.where(data['team1']==data['winner'], 1, 0))

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [85]:
a = [ExtraTreeClassifier().fit(X_train,y_train).score(X_test,y_test) for i in range(0,10000)]
print np.mean(a)
print np.max(a)
print np.min(a)

0.5222821428571428
0.8928571428571429
0.17857142857142858


In [91]:
a = [ExtraTreesClassifier().fit(X_train,y_train.values.ravel()).score(X_test,y_test) for i in range(0,10000)]
print np.mean(a)
print np.max(a)
print np.min(a)

0.5359892857142856
0.8214285714285714
0.2857142857142857


In [92]:
a = [RandomForestClassifier().fit(X_train,y_train.values.ravel()).score(X_test,y_test) for i in range(0,10000)]
print np.mean(a)
print np.max(a)
print np.min(a)

0.5401428571428571
0.7857142857142857
0.2857142857142857


In [93]:
a = [cross_val_score(ExtraTreeClassifier(), X, Y, cv=5).mean() for i in range(0,10000)]
print np.mean(a)
print np.max(a)
print np.min(a)

0.5377765277777778
0.7319444444444445
0.30277777777777776


In [53]:
cvs = cross_val_score(ExtraTreeClassifier(), X, Y, cv=5)
print "Cross Validation ExtraTreeClassifier: ", cvs.mean(), '+/-', cvs.std()

Cross Validation ExtraTreeClassifier:  0.538888888888889 +/- 0.0831711384532215
